In [40]:
import sys
sys.path.append('../koalas/')

In [59]:
import csv
lines = list(csv.reader(open('covid.csv', 'r')))
headers, rows = lines[0], lines[1:]
headers[0] = headers[0][1:]
rows = [[(0. if value == 'NA' else value) for value in row] for row in rows]

In [60]:
from koalas import DataFrame

In [61]:
df = DataFrame(fields=headers, rows=rows)

In [67]:
(
    df
        .apply('New Cases', lambda new_cases: new_cases or 0., 'new_cases')
        .rename('country', 'Country')
        .rename('year_week', 'Period')        
        .rename('population', 'Population')
        .select('Country', 'New Cases', 'Period', 'Population')      
        .apply('New Cases (Norm.)', lambda new_cases, population: round(100_000 * new_cases / population, 2), 'New Cases', 'Population')  
        .group('Period')
        .apply('Most New Cases (Norm.)', lambda new_cases: max(new_cases), 'New Cases (Norm.)')
        .apply('Country with Most New Cases (Norm.)', lambda countries, new_cases, most_new_cases: [c for c, n in zip(countries, new_cases) if n == most_new_cases][0], 'Country', 'New Cases (Norm.)', 'Most New Cases (Norm.)')
        .select('Period', 'Most New Cases (Norm.)', 'Country with Most New Cases (Norm.)')
        [:20]
)

TypeError: unsupported operand type(s) for /: 'float' and 'str'